##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Promedio de modelos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/average_optimizers_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>


## Descripción general

En este bloc de notas de muestra cómo usar el Moving Average Optimizer (optimizador de promedio móvil) junto con el Model Average Checkpoint (punto de verificación del promedio de modelos) del paquete de complementos de TensorFlow.


## Promedio móvil

> La ventaja del promedio móvil es que son menos propensos a cambios incontrolables de pérdidas o a una representación irregular de datos en el último lote. Da una idea más fluida y general del entrenamiento del modelo hasta cierto punto.

## Promedio estocástico

> El Stochastic Weight Averaging (promedio de peso estocástico, SWA) converge en óptimos más amplios. Al hacerlo, se asemeja a un conjunto geométrico. El SWA es un método simple para mejorar el rendimiento del modelo cuando se usa como contenedor de otros optimizadores y promedia los resultados de diferentes puntos de la trayectoria del optimizador interno.

## Punto de verificación promedio del modelo

> `callbacks.ModelCheckpoint` no da la opción de guardar pesos promedio móviles a la mitad del entrenamiento, por esa razón los Model Average Optimizers (optimizadores de promedio del modelo) solicitaron una retrollamada personalizada. Mediante el uso del parámetro `update_weights`, `ModelAverageCheckpoint` le permite:

1. Asignar los pesos de promedio móvil al modelo y guardarlos.
2. Mantener los pesos antiguos no promediados, pero que el modelo guardado use los pesos promedio.

## Preparación

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
import numpy as np
import os

## Construir el modelo 

In [ ]:
def create_model(opt):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),                         
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model

## Preparar el conjunto de datos

In [ ]:
#Cargar grupo de datos de Fashion MNIST 
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

Aquí comparamos tres optimizadores:

- SGD (Stochastic gradient descent [descenso de gradiente estocástico]) sin contenedor
- SGD con promedio móvil
- SGD con promedio de peso estocástico

Y a continueación puede observar cómo se desempeñan con el mismo modelo.

In [ ]:
#Optimizadores
sgd = tf.keras.optimizers.SGD(0.01)
moving_avg_sgd = tfa.optimizers.MovingAverage(sgd)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

Tanto el optimizador `MovingAverage` como el `StochasticAverage` usan `ModelAverageCheckpoint`.

In [ ]:
#Callback 
checkpoint_path = "./training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath=checkpoint_dir, 
                                                    update_weights=True)

## Entrenar el modelo


### Optimizador Vanilla SGD 

In [ ]:
#Generar modelo
model = create_model(sgd)

#Entrenar la red
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

### SGD de promedio móvil

In [ ]:
#Build Model
model = create_model(moving_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

### SGD de promedio de peso estocástico 

In [ ]:
#Generar modelo
model = create_model(stocastic_avg_sgd)

#Entrenar la red
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)